<a href="https://colab.research.google.com/github/ZhangYHe/MyColabDL_Repo/blob/main/work05_%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super(MyRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
   
        self.My_encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, dropout=1)
        self.My_decoder = nn.Linear(num_hiddens , 2)   # num_hiddens*2 ?

    def forward(self, inputs):
        # inputs [batchsize，time_step]
        # inputs_T [time_step, batchsize]
        # embeddings [time_step，batchsize，embed_size]
        # outputs [time_step，batchsize, num_hiddens]

        inputs_T=inputs.T
        embeddings = self.embedding(inputs_T)
        self.My_encoder.flatten_parameters()
        
        outputs, _ = self.My_encoder(embeddings)
        # encoding = outputs
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        type = self.My_decoder(encoding)

        return type

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])

In [ ]:
def find_gpus():
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


In [ ]:
embed_size, num_hiddens, num_layers = 100, 100, 2
devices = find_gpus()
net = MyRNN(len(vocab), embed_size, num_hiddens, num_layers)

net.apply(init_weights)

##定义网络   
网络框架如下所示：   
> input: [batchsize，time_step]  
>
> Embedding   
>> input: [batchsize，time_step]   
>> output: [time_step，batchsize，embed_size]   
>  
> LSTM
>> input: [time_step，batchsize，embed_size]  
>> output: [time_step，batchsize, num_hiddens]   
> 
>Drouout   
>> input: [time_step，batchsize, num_hiddens]  
>> output: [time_step，batchsize, num_hiddens]  
> 
> Dense   
>> input: [time_step，batchsize, num_hiddens]  
>> output: [batchsize, class_num]   
>   
> output: [batchsize, class_num]  

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super(MyRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
   
        self.My_encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, dropout=0.5)
        self.My_decoder = nn.Linear(num_hiddens*2 , 2)   # num_hiddens*2 ?

    def forward(self, inputs):
        # inputs [batchsize，time_step]
        # inputs_T [time_step, batchsize]
        # embeddings [time_step，batchsize，embed_size]
        # outputs [time_step，batchsize, num_hiddens]

        inputs_T=inputs.T
        embeddings = self.embedding(inputs_T)
        print("emb",embeddings.shape)
        self.My_encoder.flatten_parameters()
        
        outputs, _ = self.My_encoder(embeddings)
        print("outputs",outputs.shape)
        # encoding = outputs
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)

        type = self.My_decoder(encoding)
        print("type",type.shape)
        return type

##初始化模型参数   
对网络每一层的参数进行初始化

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])

In [ ]:
def find_gpus():
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


In [ ]:
embed_size, num_hiddens, num_layers = 100, 100, 2
batchsize, time_step = 5, 2
devices = find_gpus()
# net = MyRNN(len(vocab), embed_size, num_hiddens, num_layers)
net = MyRNN(28, embed_size, num_hiddens, num_layers)  #测试

net.apply(init_weights)

MyRNN(
  (embedding): Embedding(28, 100)
  (My_encoder): LSTM(100, 100, num_layers=2, dropout=0.5)
  (My_decoder): Linear(in_features=200, out_features=2, bias=True)
)

In [ ]:
#测试网络输出
X = torch.arange(10).reshape((batchsize, time_step))
Y = net(X)
X.shape, Y.shape

emb torch.Size([2, 5, 100])
outputs torch.Size([2, 5, 100])
type torch.Size([5, 2])


(torch.Size([5, 2]), torch.Size([5, 2]))